In [2]:
import yt_dlp
import concurrent.futures

In [3]:
def download_youtube_video(url, save_path="./", cookies_path="cookies.txt"):
    try:
        ydl_opts = {
            'outtmpl': f"{save_path}/%(title)s.%(ext)s",
            'format': 'bestvideo+bestaudio/best',
            # 'cookies': cookies_path  # Provide the cookies file
            'cookiesfrombrowser': ('chrome',)
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        print("Download completed!")
    except Exception as e:
        print(f"An error occurred: {e}")

In [4]:
def get_latest_videos_from_channel(channel_url, playlist_items = '1-2'):
    ydl_opts = {
        'quiet': False,  # 显示调试信息，帮助我们理解问题
        'extract_flat': True,  # 只提取视频信息，不下载视频
        'playlist_items': playlist_items, 
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            # 提取频道信息
            info_dict = ydl.extract_info(channel_url, download=False)

            # 获取视频条目列表
            videos = info_dict.get('entries', [])

            if not videos:
                print("No videos found.")
                return []

            # 提取所有视频链接
            video_links = [video['url'] for video in videos]

            return video_links

        except Exception as e:
            print(f"Error occurred: {e}")
            return []

In [5]:
def download_latest_videos_concurrently(channel_url, playlist_items='1-2'):
    """获取并并行下载频道的最新视频"""
    video_urls = get_latest_videos_from_channel(channel_url, playlist_items)
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(download_youtube_video, video_url) for video_url in video_urls]
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"An error occurred during download: {e}")

In [6]:
channel_url = 'https://www.youtube.com/@NBA/videos'
download_latest_videos_concurrently(channel_url, playlist_items='1-1')

[youtube:tab] Extracting URL: https://www.youtube.com/@NBA/videos
[youtube:tab] @NBA/videos: Downloading webpage
[download] Downloading playlist: NBA - Videos
[youtube:tab] Playlist NBA - Videos: Downloading 1 items
[download] Downloading item 1 of 1
[download] Finished downloading playlist: NBA - Videos
Extracting cookies from chrome
Extracted 3087 cookies from chrome
[youtube] Extracting URL: https://www.youtube.com/watch?v=DWb_X58aPmA
[youtube] DWb_X58aPmA: Downloading webpage
[youtube] DWb_X58aPmA: Downloading ios player API JSON
[youtube] DWb_X58aPmA: Downloading mweb player API JSON
[youtube] DWb_X58aPmA: Downloading player 2d24ba15
[youtube] DWb_X58aPmA: Downloading m3u8 information
[info] DWb_X58aPmA: Downloading 1 format(s): 399+251
[download] Destination: .//NBA’s Top 10 Plays of the Night ｜ November 18, 2024.f399.mp4
[download] 100% of   31.54MiB in 00:00:27 at 1.15MiB/s   
[download] Destination: .//NBA’s Top 10 Plays of the Night ｜ November 18, 2024.f251.webm
[download] 10

In [8]:
import subprocess

def process_video(input_file, output_file):
    """
    使用 FFmpeg 对视频进行以下处理：
    1. 移除元数据。
    2. 调整分辨率到 1280x720。
    3. 调整帧率到 30fps。
    4. 添加轻微的亮度和对比度调整。
    5. 添加随机噪声。
    """
    # 创建 FFmpeg 命令
    command = [
        "ffmpeg", "-i", input_file,  # 输入文件
        # 移除元数据
        "-map_metadata", "-1",
        # 视频滤镜：调整分辨率、帧率、亮度、对比度和添加噪声
        # "-vf", "scale=1280:720,eq=brightness=0.05:contrast=1.1,noise=alls=10:allf=t+u",
        "-vf", "scale=1280:720,eq=brightness=0.05:contrast=1.1",
        # 设置帧率
        "-r", "30",
        # 视频编码器和参数
        "-c:v", "libx264", "-crf", "18", "-b:v", "3M",
        "-preset", "slow",
        # 音频编码器
        "-c:a", "aac",
        # 输出文件
        output_file
    ]

    # 执行 FFmpeg 命令
    try:
        subprocess.run(command, check=True)
        print(f"视频处理完成，保存为: {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"处理视频时出错: {e}")

# 示例用法
if __name__ == "__main__":
    input_file = "NBA’s Top 10 Plays of the Night ｜ November 18, 2024.webm"  # 输入视频文件
    output_file = "output_processed.webm"  # 输出视频文件

    # 调用视频处理函数
    process_video(input_file, output_file)


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e

视频处理完成，保存为: output_processed.webm


[out#0/webm @ 0x14de2eaa0] video:29533KiB audio:1201KiB subtitle:0KiB other streams:0KiB global headers:4KiB muxing overhead: 0.293379%
frame= 3643 fps= 16 q=44.0 Lsize=   30824KiB time=00:02:01.43 bitrate=2079.4kbits/s dup=4 drop=0 speed=0.522x    
